In [4]:
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import numpy as np
from numpy import NaN, NAN,nan
import statistics

all_data={}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    all_data[column]=yf.download(column, start = '2014-06-02', end='2016-06-13')

luvdf=pd.read_csv('LUV.csv')
luvdf['Date'] = pd.to_datetime(luvdf['Date'])
luvdf.index = luvdf['Date']
luvdf = luvdf.drop(columns = ['Date'])
luvdf = luvdf.iloc[21:]
luvdf=luvdf.iloc[:-1]

dict1={ 'Date':np.array([k for k in all_data['AAL'].index])}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    dict1[column]=np.array([k for k in all_data[column]['Adj Close']])
price_industry = pd.DataFrame(dict1) 
price_industry.index = price_industry['Date']
price_industry = price_industry.drop(columns = ['Date'])

price_return=price_industry.pct_change(periods = 1)
price_return=price_return.replace(to_replace=[NaN],value=0)
lag=pd.DataFrame({'price_LUV':np.array([k for k in luvdf['Adj Close']]),
                  'daily_return_airline_industry' :price_return.mean(axis=1),
                'MovingAverage(5)':luvdf['Adj Close'].rolling(5).mean(),
                  'MovingAverage(25)':luvdf['Adj Close'].rolling(25).mean()
               })
lag


c:\users\naman\appdata\local\programs\python\python36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25)
Date,,,,
2014-06-02,14.711601,0.000000,NaN,NaN
2014-06-03,14.475902,0.000874,NaN,NaN
2014-06-04,14.259844,-0.005360,NaN,NaN
2014-06-05,14.289306,0.002586,NaN,NaN
2014-06-06,14.338411,0.014908,14.415013,NaN
...,...,...,...,...
2016-06-06,2.160000,0.020655,2.098000,2.2484
2016-06-07,2.170000,0.016486,2.128000,2.2488
2016-06-08,2.160000,0.008766,2.128000,2.2464


# HYPERPARAMETER_OPTIMIZATION

In [5]:
price=np.array(luvdf['Adj Close'])
maximum=0
ma_small=0
ma_big=0


for j in range(10,35,1):
    MA_BIG=np.array(luvdf['Adj Close'].rolling(j).mean())

    for i in range(2,j//2+1):
        MA_SMALL=np.array(luvdf['Adj Close'].rolling(i).mean())

        MA_small_industry=np.array(price_industry.mean(axis=1).rolling(i).mean())
        MA_big_industry=np.array(price_industry.mean(axis=1).rolling(j).mean())
        
        #INDUSTRY SIGNAL
        signals_industry=[0]
        for m in range(1,MA_small_industry.shape[0]-1):
            if MA_small_industry[m-1]==nan or MA_big_industry[m-1]==nan:
                signals_industry.append(0)
                continue
            if MA_small_industry[m-1]>=MA_big_industry[m-1] and MA_small_industry[m-2]<MA_big_industry[m-2]:
                signals_industry.append(1)
                continue
            elif MA_small_industry[m-1]<=MA_big_industry[m-1] and MA_small_industry[m-2]>MA_big_industry[m-2]:
                signals_industry.append(-1)
                continue
            else:
                signals_industry.append(0)
        
        signals=[0] 
        for k in range(1,MA_BIG.shape[0]-1):
            if MA_SMALL[k-1]==nan or MA_BIG[k-1]==nan:
                signals.append(0)
                continue
            elif (MA_SMALL[k-1]>=MA_BIG[k-1] and MA_SMALL[k-2]<MA_BIG[k-2]):
                flag1=False
                for n in range(k-1,k-9,-1):
                    if(signals_industry[n]==1):
                        signals.append(1)
                        flag1=True
                        break
                if(flag1==False):               
                    signals.append(0)


            elif (MA_SMALL[k-1]<=MA_BIG[k-1] and MA_SMALL[k-2]>MA_BIG[k-2] ):
                flag2=False
                for n in range(k-2,k-9,-1):
                    if(signals_industry[n]==-1):
                        signals.append(-1)
                        flag2=True
                        break
                if(flag2==False):               
                    signals.append(0)
            else:
                signals.append(0)
        
        signals.append(0)#last day closing all the positions so no signal generated
        signals.append(sum(signals)-0)#adding terminal wealth
        signal=np.array(signals)  
        Pnl=[-price[l] if signal[l]== 1 else +price[l] if signal[l] ==-1 else 0 for l in  range(0,len(signal)-1)]
        sumPnl=  sum(Pnl)+signal[-1]*(price[-1] )
        print(i,j,sumPnl)
        if(sumPnl>maximum):
            optimized_Pnl=Pnl
            optimized_signal=signal
            maximum=sumPnl
            ma_small=i
            ma_big=j
sumPnl=maximum
print(optimized_signal,ma_small,ma_big,maximum)


2 10 2.7127820000000034
3 10 -10.393751000000007
4 10 -20.257255999999995
5 10 5.252099000000001
2 11 -1.7472179999999984
3 11 -17.291992999999998
4 11 -8.415382
5 11 6.892098999999996
2 12 -16.180694000000003
3 12 -16.211993000000003
4 12 -2.6399999999999992
5 12 -16.302744000000004
6 12 -13.642046000000006
2 13 -17.330694
3 13 -19.022422
4 13 -14.052743999999997
5 13 -4.424306000000001
6 13 4.336189000000001
2 14 -3.678700999999999
3 14 9.358564
4 14 -34.29946100000001
5 14 -19.699301000000006
6 14 -8.638806
7 14 21.900860999999995
2 15 2.0812989999999987
3 15 8.518563999999996
4 15 -13.436717000000002
5 15 -31.093479000000002
6 15 -4.673111999999996
7 15 -13.879553000000014
2 16 9.818779999999997
3 16 9.858564
4 16 -6.884645
5 16 -16.399301
6 16 -15.922665000000004
7 16 -16.48930200000001
8 16 0.42999899999999736
2 17 5.168779999999998
3 17 14.207130999999997
4 17 -1.7346450000000018
5 17 -3.9831120000000007
6 17 -1.413111999999999
7 17 -12.319302
8 17 0.35892399999999824
2 18 -6.60

In [6]:
lag['SIGNAL']=np.array(optimized_signal[:-1])
lag['P/L']=np.array(optimized_Pnl)
lag

,price_LUV,daily_return_airline_industry,MovingAverage(5),MovingAverage(25),SIGNAL,P/L
Date,,,,,,
2014-06-02,14.711601,0.000000,NaN,NaN,0,0.0
2014-06-03,14.475902,0.000874,NaN,NaN,0,0.0
2014-06-04,14.259844,-0.005360,NaN,NaN,0,0.0
2014-06-05,14.289306,0.002586,NaN,NaN,0,0.0
2014-06-06,14.338411,0.014908,14.415013,NaN,0,0.0
...,...,...,...,...,...,...
2016-06-06,2.160000,0.020655,2.098000,2.2484,0,0.0
2016-06-07,2.170000,0.016486,2.128000,2.2488,0,0.0
2016-06-08,2.160000,0.008766,2.128000,2.2464,0,0.0


In [7]:
days_when_received_signal=[]
for i in range(len(optimized_signal)):
    if optimized_signal[i]!=0:
        days_when_received_signal.append(i)
print(days_when_received_signal)

[68, 77, 150, 194, 218, 236, 263, 282, 325, 343, 351, 392, 427, 478, 486, 512]


# volatility 

In [8]:
def volatility_calculator(optimized_signal,sumPnl):
    adj_close_luv=price
    adj_close_return=np.diff(adj_close_luv)
    vol=[]
    buy=False
    sell=False
    for i in range(len(adj_close_return)):
        if optimized_signal[i]==1:
            if sell==True:
                buy=False
                sell=False
                vol.append(adj_close_return[i])
            else:
                buy=True
                sell=False
        if optimized_signal[i]==-1:
            if buy==True:
                vol.append(adj_close_return[i])
                buy=False
                sell=False
            else:
                buy=False
                sell=True
        if buy==False and sell==False:
            continue
        if buy==True and sell==False:
            vol.append(adj_close_return[i])
        if buy==False and sell==True:
            vol.append(adj_close_return[i])
            
    if optimized_signal[-1]!=0:
        vol.append(adj_close_return[-1])

    print("mean return=",sumPnl/(len(days_when_received_signal)-1),"volatility=",np.array(vol).std(),"sharperatio=",(sumPnl/(len(days_when_received_signal)-1))/np.array(vol).std())


In [9]:
volatility_calculator(optimized_signal,sumPnl)

mean return= 1.4600573999999997 volatility= 0.328048949076361 sharperatio= 4.450730307507059


## is my strategy better? following are the results of using simple crossover strategy for LUV stock witout considering lag 

In [10]:
MA_small=np.array(luvdf['Adj Close'].rolling(8).mean())
MA_large=np.array(luvdf['Adj Close'].rolling(20).mean())
signal=[]
for i in range(1,MA_small.shape[0]-1):
    if MA_small[i-1]==nan or MA_large[i-1]==nan:
        signal.append(0)
        continue
    if MA_small[i-1]>=MA_large[i] and MA_small[i-2]<MA_large[i-2]:
        signal.append(1)
        continue
    elif MA_small[i-1]<=MA_large[i-1] and MA_small[i-2]>MA_large[i-2]:
        signal.append(-1)
        continue
    else:
        signal.append(0)
        
signal.append(0)     
signal.append(sum(signal)-0)
signal=np.array(signal)  
Pnl=[-price[l] if signal[l]== 1 else +price[l] if signal[l] ==-1 else 0 for l in  range(0,len(signal)-1)]
sumPnl= sum(Pnl)+signal[-1]*(price[-1])
print(signal,sumPnl)
days_when_received_signal=[]
for i in range(len(signal)):
    if signal[i]!=0:
        days_when_received_signal.append(i)
print(days_when_received_signal)
volatility_calculator(signal,sumPnl)


[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0

## TRADING SYSYTEM

In [11]:
import datetime
import pandas as pd
import numpy as np

class trading_strategy:


    def __init__(self):

        self.todays_date = datetime.datetime.now().date()
        self.index = pd.date_range(self.todays_date, periods=1)
        self.columns = ['AdjClose_luv','AdjClose_industry']
        self.df= pd.DataFrame(index=self.index, columns=self.columns)
        self.df=self.df.drop(self.df.index[0])
        self.moving_average_luv_small=[]
        self.moving_average_luv_big=[]
        self.moving_average_industry_small=[]
        self.moving_average_industry_big=[]
        
        self.pnl=0
        self.sumsignal=0
    
    def process_tick(self, adujusted_price,industry_price):
        
        self.todays_date+=datetime.timedelta(days=1)
        row = pd.Series({'AdjClose':adujusted_price,"Adj_close_industry":industry_price},name=self.todays_date)
        self.df = self.df.append(row)
        
        self.moving_average_luv_small=np.array(self.df['AdjClose'].rolling(7).mean())
        self.moving_average_luv_big=np.array(self.df['AdjClose'].rolling(14).mean())
        self.moving_average_industry_small=np.array(self.df['Adj_close_industry'].rolling(7).mean())
        self.moving_average_industry_big=np.array(self.df['Adj_close_industry'].rolling(14).mean())
        
        signal_received=self.check_signal()
        
        self.sumsignal+=signal_received
        if(signal_received==1):
            self.generate_buy_order(adujusted_price)
        
        elif(signal_received==-1):
            self.generate_sell_order(adujusted_price)
            
        self.update_pnl(signal_received,adujusted_price)
   
            
    def generate_buy_order(self,adjusted_price):
        print("BUY",adjusted_price,self.todays_date)
    def generate_sell_order(self,adjusted_price):
        print("SELL",adjusted_price,self.todays_date)   
        
    def check_industry_signal(self,k):
        signals_industry=[0]
        for m in range(1,k-1):
            if self.moving_average_industry_small[m-1]==nan or self.moving_average_industry_big[m-1]==nan:
                signals_industry.append(0)
                continue
            if self.moving_average_industry_small[m-1]>=self.moving_average_industry_big[m-1] and self.moving_average_industry_small[m-2]<self.moving_average_industry_big[m-2]:
                signals_industry.append(1)
                continue
            elif self.moving_average_industry_small[m-1]<=self.moving_average_industry_big[m-1] and self.moving_average_industry_small[m-2]>self.moving_average_industry_big[m-2]:
                signals_industry.append(-1)
                continue
            else:
                signals_industry.append(0)
        signals_industry.append(0)
        return signals_industry
        
    def check_signal(self):
        k=len(self.moving_average_luv_small)
        signals_industry=self.check_industry_signal(k)
         
        if self.moving_average_luv_small[k-1]==nan or self.moving_average_luv_big[k-1]==nan:
            return 0
         
        elif self.moving_average_luv_small[k-1]>=self.moving_average_luv_big[k-1] and self.moving_average_luv_small[k-2]<self.moving_average_luv_big[k-2]:
            for n in range(k-1,k-9,-1):
                if(signals_industry[n]==1):
                    return 1
            return 0
        elif self.moving_average_luv_small[k-1]<=self.moving_average_luv_big[k-1] and self.moving_average_luv_small[k-2]>self.moving_average_luv_big[k-2]:
            for n in range(k-2,k-9,-1):
                if(signals_industry[n]==-1):
                    return -1
            return 0
        
        else:
            return 0     
        
    def update_pnl(self,signal_received,adujusted_price):
        if(signal_received==1):
            self.pnl-=adujusted_price
        elif(signal_received==-1):
            self.pnl+=adujusted_price
        else:
            self.pnl+=0
    def display_pnl(self):
        print("pnl=",self.pnl+(self.sumsignal-0)*self.df['AdjClose'].iloc[-1])


# VALIDATION OF WORKING OF ABOVE CLASS

In [12]:
ts_new=trading_strategy()
list1=luvdf['Adj Close'].values
list2=price_industry.mean(axis=1)

ts_new=trading_strategy()
for i in range(len(list1)):
    ts_new.process_tick(list1[i],list2[i])

ts_new.display_pnl()

SELL 14.324132999999998 2020-07-31
SELL 13.097481 2020-08-09
BUY 7.52 2020-10-21
SELL 5.92 2020-12-04
BUY 5.22 2020-12-28
SELL 5.93 2021-01-15
BUY 5.28 2021-02-11
SELL 5.14 2021-03-02
BUY 3.5 2021-04-14
SELL 3.04 2021-05-02
BUY 3.59 2021-05-10
SELL 3.15 2021-06-20
BUY 1.95 2021-07-25
BUY 2.39 2021-09-14
SELL 2.34 2021-09-22
pnl= 21.421613999999995


# BACKTESTING

In [14]:
luvdf=yf.download("LUV", start = '2017-06-02', end='2019-06-13')
list1=luvdf['Adj Close'].values

all_data={}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    all_data[column]=yf.download(column, start = '2017-06-02', end='2019-06-13')
dict1={ 'Date':np.array([k for k in all_data['AAL'].index])}
for column in ['AAL','ALK','AVH','CEA','ZNH','VLRS','CPA','DAL','GOL','LTM','UAL']:
    dict1[column]=np.array([k for k in all_data[column]['Adj Close']])
price_industry = pd.DataFrame(dict1) 
price_industry.index = price_industry['Date']
price_industry = price_industry.drop(columns = ['Date'])
list2=price_industry.mean(axis=1)

print(len(list1),len(list2))    
ts_new2=trading_strategy()
for i in range(len(list1)):
    ts_new2.process_tick(list1[i],list2[i])

ts_new2.display_pnl()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
510 510
BUY 52.00429916381836 2020-08-02
SELL 53.469356536865234 2020-09-04
SELL 56.109039306640625 2020-12-15
BUY 50.99614334106445 2021-02-10
SELL 49.54971694946289 2021-